# Preliminary

## Install package

In [ ]:
!pip install wandb -q

In [ ]:
!pip install python-Levenshtein
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..
!pip install torchsummaryX

## Import Package

In [ ]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import ctcdecode
import Levenshtein
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


## Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Kaggle setup

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"","key":""}') # TODO: Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c 11-785-f22-hw3p2

100% 8.87G/8.88G [00:54<00:00, 111MB/s] 
100% 8.88G/8.88G [00:54<00:00, 176MB/s]


In [ ]:
'''
This will take a couple minutes, but you should see at least the following:
11-785-f22-hw3p2.zip  ctcdecode  hw3p2
'''
!unzip -q 11-785-f22-hw3p2.zip
!ls

11-785-f22-hw3p2.zip  ctcdecode  hw3p2	sample_data


# Dataset

In [ ]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE
# This overwrites the phonetics.py file.
CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"}
CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())

PHONEMES = CMUdict
mapping = CMUdict_ARPAbet
LABELS = ARPAbet

In [ ]:
print(len(LABELS))

43


## TrainData

In [ ]:
class AudioDataset(torch.utils.data.Dataset):
    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW
    #TODO
    def __init__(self, data_path, partition= "train-clean-100", limit=-1): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        ''' 
        # Load the directory and all files in them
        self.data_path = data_path

        self.mfcc_dir = self.data_path +'/'+ partition + '/mfcc' #TODO
        self.transcript_dir = self.data_path +'/'+ partition + '/transcript/raw' #TODO

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir)) #TODO
        self.transcript_files = sorted(os.listdir(self.transcript_dir)) #TODO

        assert len(self.mfcc_files) == len(self.transcript_files) 
        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        CMUdict_ARPAbet = {
              "" : " ",
              "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
              "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
              "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
              "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
              "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
              "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
              "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
              "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
              "[SOS]": "[SOS]", "[EOS]": "[EOS]"}
        CMUdict = list(CMUdict_ARPAbet.keys())
        ARPAbet = list(CMUdict_ARPAbet.values())
        self.PHONEMES = CMUdict
        self.mapping = CMUdict_ARPAbet
        self.LABELS = ARPAbet
        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs, self.transcripts = [], []
        for i in range(0, len(self.mfcc_files)):
            mfcc = np.load(self.mfcc_dir + '/' + self.mfcc_files[i])
        #   Gaussin Normalization of mfcc
            mean = np.mean(mfcc,axis = 0)
            sigma = np.std(mfcc, axis = 0)
            mfcc = (mfcc - mean)/sigma
        #   Load the corresponding transcript
        #   Remove [SOS] and [EOS] from the transcript 
            transcript = np.load(self.transcript_dir + '/' + self.transcript_files[i])
            transcript = transcript[1: -1]
            # Map to label
            transcript = [self.LABELS.index(self.mapping[transcript[index]]) for index in range(len(transcript))]
            # Transform to numerical
            #transcript = np.array([transcript.index(self.LABELS[i]) for i in range(len(transcript))])
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)
            # self.label.append(label)
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       
    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS
        If you didn't do the loading and processing of the data in __init__,
        do that here.
        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.Tensor(self.mfccs[ind])# TODO
        transcript = torch.Tensor(self.transcripts[ind])# TODO
        #abel = self.labels[ind]
        return mfcc, transcript

    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        # batch of output phonemes
        batch_mfcc, batch_transcript = zip(*batch)# TODO
        #batch_transcript = [transcript for mfcc, transcript in batch] # TODO
        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0) # TODO
        lengths_mfcc =  torch.LongTensor([len(x) for x in batch_mfcc])# TODO 

        batch_transcript_pad = pad_sequence(batch_transcript, batch_first=True, padding_value=0) # TODO
        lengths_transcript =  torch.LongTensor([len(x) for x in batch_transcript])# TODO

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)

       

## TestData

In [ ]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW
    #TODO
    def __init__(self, data_path, partition= "test-clean", limit=-1): 
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        ''' 
        # Load the directory and all files in them
        self.data_path = data_path
        self.mfcc_dir = self.data_path +'/'+ partition + '/mfcc' #TODO
        self.mfcc_files = sorted(os.listdir(self.mfcc_dir)) #TODO
        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)
        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        CMUdict_ARPAbet = {
          "" : " ",
          "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@", 
          "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W", 
          "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R", 
          "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w", 
          "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y", 
          "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D", 
          "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O", 
          "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
          "[SOS]": "[SOS]", "[EOS]": "[EOS]"}
        CMUdict = list(CMUdict_ARPAbet.keys())
        ARPAbet = list(CMUdict_ARPAbet.values())
        self.PHONEMES = CMUdict
        self.mapping = CMUdict_ARPAbet
        self.LABELS = ARPAbet
        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        self.mfccs = []
        for i in range(0, len(self.mfcc_files)):
            mfcc = np.load(self.mfcc_dir + '/' + self.mfcc_files[i])
        #   Gaussin Normalization of mfcc
            mean = np.mean(mfcc,axis = 0)
            sigma = np.std(mfcc, axis = 0)
            mfcc = (mfcc - mean)/sigma
        #   Append each mfcc to self.mfcc
            self.mfccs.append(mfcc)
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
       
    def __len__(self):
        
        '''
        TODO: What do we return here?
        '''
        return self.length
        # raise NotImplemented

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS
        If you didn't do the loading and processing of the data in __init__,
        do that here.
        Once done, return a tuple of features and labels.
        '''
        mfcc = torch.Tensor(self.mfccs[ind])# TODO
        return mfcc

    def collate_fn(batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish. 
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features, 
            and lengths of labels.
        '''
        # batch of input mfcc coefficients
        # batch of output phonemes
        batch_mfcc = batch # TODO
        #batch_transcript = [transcript for mfcc, transcript in batch] # TODO
        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first=True, padding_value=0.0) # TODO
        lengths_mfcc =  torch.LongTensor([len(x) for x in batch_mfcc]) # TODO 

        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?
        
        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)

## Data Hyperparameter

In [ ]:
transforms = [] # set of tranformations
# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
config = {'batch_size': 64}
root = '/content/hw3p2' 

## Dataloader

In [ ]:
# get me RAMMM!!!! 
import gc 
gc.collect()

516

In [ ]:
# Create objects for the dataset class
train_data = AudioDataset(root, partition= "train-clean-360", limit=-1) #TODO
val_data = AudioDataset(root, partition= "dev-clean", limit=-1) # TODO : You can either use the same class with some modifications or make a new one :)
test_data = AudioDatasetTest(root, partition= "test-clean", limit=-1) #TODO

# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(train_data, 
                                           collate_fn=AudioDataset.collate_fn,
                                           num_workers= 4, 
                                           batch_size=config['batch_size'], 
                                           pin_memory= True,shuffle=True
                                           )
val_loader = torch.utils.data.DataLoader(val_data, 
                                         collate_fn=AudioDataset.collate_fn,
                                         num_workers= 4, 
                                         batch_size=config['batch_size'], 
                                         pin_memory= True,shuffle=False
                                         )#TODO
test_loader = torch.utils.data.DataLoader(test_data, 
                                          collate_fn=AudioDatasetTest.collate_fn,
                                          num_workers= 4, 
                                          batch_size=config['batch_size'], 
                                          pin_memory= True,shuffle=False
                                          )#TODO

## Sanity Check

In [ ]:
print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 104014, batches = 1626
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 41


In [ ]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx, ly)
    break 

# Model Config

In [ ]:
OUT_SIZE = len(LABELS)

43

## Basic

In [ ]:
class GaussianNoise(nn.Module):
    """Gaussian noise regularizer.

    Args:
        sigma (float, optional): relative standard deviation used to generate the
            noise. Relative means that it will be multiplied by the magnitude of
            the value your are adding the noise to. This means that sigma can be
            the same regardless of the scale of the vector.
        is_relative_detach (bool, optional): whether to detach the variable before
            computing the scale of the noise. If `False` then the scale of the noise
            won't be seen as a constant but something to optimize: this will bias the
            network to generate vectors with smaller values.
    """

    def __init__(self, sigma=0.1, is_relative_detach=True):
        super().__init__()
        self.sigma = sigma
        self.is_relative_detach = is_relative_detach
        self.noise = torch.tensor(0).to(device)

    def forward(self, x):
        if self.training and self.sigma != 0:
            scale = self.sigma * x.detach() if self.is_relative_detach else self.sigma * x
            sampled_noise = self.noise.repeat(*x.size()).normal_() * scale
            x = x + sampled_noise
        return x 

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels = 15, out_channels = 64, stride = 1, dropout = 0.1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Sequential(
                        nn.ConstantPad1d(3 // 2, 0),
                        nn.Conv1d(in_channels, out_channels, kernel_size = 3, stride = stride, bias = False),
                        nn.BatchNorm1d(out_channels),
                        nn.ReLU(),
                        torch.nn.Dropout(dropout),

                        nn.ConstantPad1d(3 // 2, 0),
                        nn.Conv1d(out_channels, out_channels, kernel_size = 3, stride = 1, bias = False),
                        nn.BatchNorm1d(out_channels),
                        torch.nn.Dropout(dropout)
                        )

        self.downsample = downsample
        # self.downsample = nn.Sequential(
        #         nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1, bias = False),
        #         nn.BatchNorm1d(64),
        #     )
        self.relu = nn.ReLU()
        self.out_channels = out_channels
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [ ]:
class Class(nn.Module):
    def __init__(self, insize = 512, num_classes = 43, dropout = 0.1):
        super(Class, self).__init__()
        self.linear1 = nn.Linear(insize, 1024, bias = False)
        self.batchnorm1 = nn.BatchNorm1d(1024)

        self.linear2 = nn.Linear(1024, 2048, bias = False)
        self.batchnorm2 = nn.BatchNorm1d(2048)

        self.linear3 = nn.Linear(2048, num_classes)

        self.relu = nn.ReLU()
        self.drop = nn.Dropout(p = dropout)

        
    def forward(self, x):
        x = self.linear1(x)
        x = x.transpose(1,2)
        x = self.batchnorm1(x)
        x = x.transpose(1,2)
        x = self.relu(x)
        x = self.drop(x)

        x = self.linear2(x)
        x = x.transpose(1,2)
        x = self.batchnorm2(x)
        x = x.transpose(1,2)
        x = self.relu(x)
        x = self.drop(x)

        x = self.linear3(x)


        return x

In [ ]:
from torch.nn.modules import dropout
torch.cuda.empty_cache()

class Network(nn.Module):
    def __init__(self, block, Class, num_classes = 43, embed_drop = 0.1, cls_dropout = 0.1,lstm_drop = 0.1, layer = 1):
        super(Network, self).__init__()
        # Adding some sort of embedding layer or feature extractor might help performance.
        #self.embedding = torch.nn.Embedding(num_embeddings = , embedding_dim = 15, padding_idx = 0)
        self.embedding = self.make_layer(block, 15, 64, block_num = layer, stride = 1, drop = embed_drop)
        # self.embedding1 = self.make_layer(block, 64, 128, block_num = layer, stride = 1, drop = embed_drop)
        # self.embedding = block(15, 64, stride = 1, dropout = embed_drop)
        # self.cnn1 = nn.Conv1d(in_channels = 15, out_channels = 64, kernel_size = 3, stride = 1, padding = 1)
        # self.batchnorm = nn.BatchNorm1d(64)
        # # TODO : look up the documentation. You might need to pass some additional parameters.
        self.lstm = nn.LSTM(input_size = 64, hidden_size = 256, num_layers = 4, batch_first = True, dropout = lstm_drop, bidirectional = True) 
        self.cls = Class(insize = 512, num_classes = num_classes, dropout = cls_dropout)
        self.logSoftmax = torch.nn.LogSoftmax(2)#TODO: Apply a log softmax here. Which dimension would apply it on ?

    def make_layer(self, block, in_channel, out_channel, block_num, stride=1, drop=0.3):
        downsample = None
        if stride != 1 or in_channel != out_channel:
            downsample = nn.Sequential(
                 nn.Conv1d(in_channel, out_channel, kernel_size=1, stride=1, bias = False),
                 nn.BatchNorm1d(out_channel),
            )
        layers = []
        layers.append(block(in_channel, out_channel, stride, drop, downsample))
        in_channel = out_channel
        for i in range(1, block_num):
            layers.append(block(in_channel, out_channel, stride, drop))

        return nn.Sequential(*layers)

    def forward(self, x, lx):
        #TODO
        # The forward function takes 2 parameter inputs here. Why?
        # Refer to the handout for hints
        x = self.embedding(x.transpose(1,2))
        x = x.transpose(1,2)
        # x = self.embedding1(x.transpose(1,2))
        # x = x.transpose(1,2)
        packed_x = pack_padded_sequence(x, lx, batch_first = True,  enforce_sorted=False)
        packed_out = self.lstm(packed_x)[0]
        out, out_lens = pad_packed_sequence(packed_out, batch_first=True)
        # # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.cls(out)
        out = self.logSoftmax(out)
        return out, out_lens

# Config

In [ ]:
run.finish()

In [ ]:
config = {
    "beam_width" : 2,
    "lr" : 2e-3,
    "epochs" : 50,
    'embed_drop' : 0.6,
     'cls_dropout' : 0.6,
     'lstm_dropout' : 0.6,
     'cnn_layer' : 4,
     'weight_decay' : 5e-5,
     'model': 'lstm1141'
    } # Feel free to add more items here

## INIT

In [ ]:
model = Network(block = ResidualBlock, Class = Class, num_classes = 43, embed_drop = config['embed_drop'], cls_dropout = config['cls_dropout'], lstm_drop = config['lstm_dropout'], layer = config['cnn_layer']).to(device)
summary(model, x.to(device), lx) # x and lx come from the sanity check above :)

# Training Config

In [ ]:
#TODO
criterion = nn.CTCLoss(blank = 0)# Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints
optimizer =  torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay']) # What goes in here?
# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(
    LABELS,
    alpha=0,
    beta=0,
    cutoff_top_n=40,
    cutoff_prob=1.0,
    beam_width=config['beam_width'],
    num_processes=4,
    blank_id=0,
    log_probs_input=True
)#TODO 
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience = 2, factor = 0.8)#TODO
# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

In [ ]:
import wandb
wandb.login(key="")
run = wandb.init(
    name = config['model'], ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    #project="Shuxian", entity="hw2_p2",
    project = 'hw3', entity = '',
    config = config)

In [ ]:
artifact = run.use_artifact('', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact lstm1141:v43, 93.90MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.4


In [ ]:
model.load_state_dict(torch.load('')['model_state_dict'])
optimizer.load_state_dict(torch.load('')['optimizer_state_dict'])

### Levenshtein

In [ ]:
# Use debug = True to see debug outputs
def calculate_levenshtein(h, y, lh, ly, decoder, labels, debug = False):
    if debug:
        pass
        # print(f"\n----- IN LEVENSHTEIN -----\n")
        # Add any other debug statements as you may need
        # you may want to use debug in several places in this function
        
    # TODO: look at docs for CTC.decoder and find out what is returned here
    beam_results, _, _, out_lens = decoder.decode(h, seq_lens = lh)
    batch_size = y.shape[0] # TODO
    distance = 0 # Initialize the distance to be 0 initially

    for i in range(batch_size): 
        # TODO: Loop through each element in the batch
        y_seq = y[i, :int(ly[i])]
        y_pron = ''.join(labels[int(i)] for i in y_seq)
        predict_y_seq = beam_results[i,0, :out_lens[i,0]]
        predict_y_pron = ''.join(labels[int(i)] for i in predict_y_seq)
        d = Levenshtein.distance(y_pron, predict_y_pron)
        distance += d
        pass
    distance /= batch_size # TODO: Uncomment this, but think about why we are doing this
    return distance

# Training

### Eval function
Writing a function to do one round of evaluations will help make your code more modular, you can, however, choose to skip this if you'd like it.

In [ ]:
def evaluate(data_loader, model):
    model.eval()
    dist = 0
    loss = 0
     batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Val') 
    val_loss = 0
    # TODO Fill this function out, if you're using it.
    for i, data in enumerate(data_loader):
        # TODO: Fill this with the help of your sanity check
        x, y, lx, ly = data
        x = x.to(device)
        y = y.to(device)
        with torch.no_grad():
          out, out_lens = model(x, lx)
          loss = criterion(out.transpose(0,1), y, out_lens, ly)
          d = calculate_levenshtein(out, y, out_lens, ly, decoder, LABELS, debug = False)
        val_loss += loss
        dist += d
        # HINT: Are you using mixed precision? 
        batch_bar.set_postfix(
            loss = f"{val_loss/ (i+1):.4f}",
            distance = f"{dist/(i+1):.4f}"
        )

        batch_bar.update()
    
    batch_bar.close()
    val_loss /= len(data_loader)
    dist /= len(data_loader) # TODO
    return val_loss, dist

### Training Setup

In [ ]:
# This is for checkpointing, if you're doing it over multiple sessions
last_epoch_completed = 0
start = last_epoch_completed
end = config['epochs']
best_val_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.
dist_freq = 1

Again, writing a train step might help you code be more modular. You may choose to skip this and write the whole thing out in the training loop below if you so wish.

In [ ]:
def train_step(train_loader, model, optimizer, criterion, scheduler, scaler):
    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 
    train_loss = 0
    dist = 0
    for i, data in enumerate(train_loader):
        # TODO: Fill this with the help of your sanity check
        optimizer.zero_grad()
        x, y, lx, ly = data
        x = x.to(device)
        y = y.to(device)
        with torch.cuda.amp.autocast():
          out, out_lens = model(x, lx)
          loss = criterion(out.transpose(0,1), y, out_lens, ly)
          #d = calculate_levenshtein(out, y, out_lens, ly, decoder, LABELS, debug = False)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        # HINT: Are you using mixed precision? 
        batch_bar.set_postfix(
            loss = f"{train_loss/ (i+1):.4f}",
            #dist = f"{dist/ (i+1):.4f}",
            lr = f"{optimizer.param_groups[0]['lr']}"
        )

        train_loss += loss
        #dist += d
        batch_bar.update()
    
    batch_bar.close()
    train_loss /= len(train_loader) # TODO
    #dist /= len(train_loader)
    return train_loss#, dist # And anything else you may wish to get out of this function

### Train Loop

In [ ]:
import wandb
wandb.login(key="") 

In [ ]:
run = wandb.init(
    name = config['model'], ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    project = '', entity = '',
    config = config)

In [ ]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
file_write = arch_file.write("\n")
file_write = arch_file.write("parameter setting:\n")
for key, value in config.items(): 
        arch_file.write('%s:%s\n' % (key, value))
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

In [ ]:
torch.cuda.empty_cache()
gc.collect()

460

In [ ]:
#TODO: Please complete the training loop
for epoch in range(config["epochs"]):
    # one training step
    train_loss = train_step(train_loader, model, optimizer, criterion, scheduler, scaler)
    # one validation step (if you want)
    torch.cuda.empty_cache()
    gc.collect()
    val_loss, val_dist = evaluate(val_loader, model)
    scheduler.step(val_loss)
    # HINT: Calculating levenshtein distance takes a long time. Do you need to do it every epoch?
    # Does the training step even need it? 
    print("\nEpoch {}/{}: \t Train Loss {:.04f} ".format(
          epoch + 1,
          config['epochs'],
          train_loss,
          #train_dist
          ))
    print("Val dist {:.04f}\t Val Loss {:.04f}\t ".format(val_dist, val_loss))

    wandb.log({"train_loss":train_loss, 
                'validation_dist':val_dist, 'validation_loss': val_loss,
               'lr' : optimizer.param_groups[0]['lr']})
    # Where you have your scheduler.step depends on the scheduler you use.

    # Use the below code to save models
    # if val_dist < best_val_dist:
    #   #path = os.path.join(root_path, model_directory, 'checkpoint' + '.pth')
    #   print("Saving model")
    #   torch.save({'model_state_dict':model.state_dict(),
    #               'optimizer_state_dict':optimizer.state_dict(),
    #               'val_dist': val_dist, 
    #               'epoch': epoch}, './checkpoint.pth')
    #   best_val_dist = val_dist
    #   wandb.save('checkpoint.pth')
    if val_dist < best_val_dist:
        #path = os.path.join(root, model_directory, 'checkpoint' + '.pth')
        print("Saving model")
        # Saving the model and optimizer states
        torch.save({
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              #'scheduler_state_dict':scheduler.state_dict()
              'val_dist': val_dist, 
               'epoch': epoch
              }, "Model")
        # Creating Artifact
        model_artifact = wandb.Artifact(config['model'], type='model')
        # Adding model file to Artifact
        model_artifact.add_file("Model")
        # Saving Artifact to WandB
        run.log_artifact(model_artifact)
        best_val_dist = val_dist

run.finish()

In [ ]:
run.finish()

In [ ]:
train_loss, train_dist = evaluate(train_loader, model)
print(train_loss)
print(train_dist)

In [ ]:
val_dist

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder
decoder_test = CTCBeamDecoder(LABELS,
                            alpha=0,
                            beta=0,
                            cutoff_top_n=40,
                            cutoff_prob=1.0,
                            beam_width=5,
                            num_processes=4,
                            blank_id=0,
                            log_probs_input=True)

In [ ]:
def predict(test_loader, model, decoder, LABELS):
    model.eval()
    df = pd.DataFrame(columns = ["id","Predicted"])
    predicted = []
    for i, data in enumerate(test_loader):
        # TODO: Fill this with the help of your sanity check
        x, lx = data
        x = x.to(device)
        # lx = lx.to(device)
        with torch.no_grad():
          out, out_lens = model(x, lx)
        beam_results, beam_scores, timesteps, out_seq_len = decoder_test.decode(out, seq_lens = out_lens) #TODO: What parameters would the decode function take in?
        batch_size = out.shape[0]#What is the batch size
        for i in range(batch_size): # Loop through each element in the batch
          h_sliced = beam_results[i, 0, : out_seq_len[i,0]]#TODO: Obtain the beam results
          h_string = ''.join(LABELS[int(i)]for i in h_sliced)#TODO: Convert the beam results to phonemes
          predicted.append(h_string)
    return predicted

In [ ]:
#TODO:
# Write a function (predict) to generate predictions and submit the file to Kaggle
torch.cuda.empty_cache()
predictions = predict(test_loader, model, decoder, LABELS)

In [ ]:
import pandas as pd
df = pd.DataFrame()
df['index'] = [i for i in range(len(predictions))]
df['label'] = predictions
df.to_csv('submission.csv', index = False)

In [ ]:
!kaggle competitions submit -c <competition> -f 'submission.csv' -m "I made it!"

/bin/bash: competition: No such file or directory
